# Imports

In [ ]:
import os
import json
import matplotlib.pyplot as plt
from PIL import Image
from collections import Counter

In [ ]:
import matplotlib
matplotlib.rcParams['figure.facecolor'] = 'white'

In [ ]:
%run ../iu_xray.py

In [ ]:
REPORTS_DIR = os.path.join(DATASET_DIR, 'reports')

In [ ]:
DICOM_DIR = '/mnt/data/iu-x-ray/dicoms'

# Load DICOMs

In [ ]:
import pydicom as pydcm

In [ ]:
%run ../../utils/dicom.py

In [ ]:
# report_id, image_id = '1798', 'IM-0518-1001'
report_id, image_id = '3179', 'IM-1499-1001'
dicom_fpath = os.path.join(DICOM_DIR, report_id, f'{report_id}_{image_id}.dcm')
png_fpath = os.path.join(DATASET_DIR, 'images', f'CXR{report_id}_{image_id}.png')

In [ ]:
dicom = pydcm.dcmread(dicom_fpath)
# dicom

In [ ]:
image_png = load_image(png_fpath, image_format='L')
image_png = np.array(image_png)
image_png.shape, image_png.dtype

In [ ]:
image_dicom = dicom_to_np(dicom, voi_lut=True).astype(np.uint16)
image_dicom.dtype, image_dicom.shape

In [ ]:
def stats(arr):
    print(arr.dtype, arr.min(), arr.max(), arr.mean(), arr.std())

In [ ]:
stats(image_png)
stats(image_dicom)

In [ ]:
n_rows, n_cols = 1, 2
plt.figure(figsize=(14, 5))

plt.subplot(n_rows, n_cols, 1)
plt.title('PNG')
plt.imshow(image_png, cmap='gray')

plt.subplot(n_rows, n_cols, 2)
plt.title('Dicom')
plt.imshow(image_dicom, cmap='gray')

In [ ]:
out_fpath = png_fpath.replace('/images/', '/images-16bit/')

In [ ]:
image = Image.fromarray(image_dicom, mode='I;16')
image = image.resize((1024, 1024))
image.size, image.mode

In [ ]:
plt.imshow(image, cmap='gray')

In [ ]:
image.save(out_fpath, optimize=True)

In [ ]:
from glob import glob

In [ ]:
png_names = glob(f'{DATASET_DIR}/images/*.png')
len(png_names)

In [ ]:
dicom_names = glob(f'{DICOM_DIR}/*/*.dcm')
len(dicom_names)

In [ ]:
dicom_names[:2]

In [ ]:
def name_dicom_to_png(dicom_path):
    dicom_name = os.path.basename(dicom_path)
    if dicom_name.startswith('1_'):
        # Special case for this patient
        dicom_name = '1_' + dicom_name
    return 'CXR' + dicom_name.replace('.dcm', '.png')

In [ ]:
future_pngs = set([name_dicom_to_png(d) for d in dicom_names])
current_pngs = set([os.path.basename(p) for p in png_names])
future_pngs == current_pngs

In [ ]:
from tqdm import tqdm

In [ ]:
dtypes = dict()

for dicom_path in tqdm(dicom_names):
    dicom = pydcm.dcmread(dicom_path)
    
    dtypes[dicom_path] = dicom.pixel_array.dtype
    
    arr = dicom_to_np(dicom, voi_lut=True)
    arr = arr.astype(np.uint16) # np.array
    
    image = Image.fromarray(arr, mode='I;16')
    image = image.resize((1024, 1024))
    
    out_path = os.path.join(DATASET_DIR, 'images-16bit-1024p', name_dicom_to_png(dicom_path))
    image.save(out_path)
    
Counter(dtypes.values()) # all IU dicoms come from uint16

In [ ]:
# path = '/mnt/workspace/iu-x-ray/dataset/images-16bit/CXR1503_IM-0329-5001.png'
# path = '/mnt/workspace/iu-x-ray/dataset/images-16bit/CXR339_IM-1635-2001.png'
path = out_path
image_loaded = Image.open(path).convert('I;16')

In [ ]:
plt.figure(figsize=(15, 5))
n_rows, n_cols = 1, 3

plt.subplot(n_rows, n_cols, 1)
plt.imshow(image_loaded, cmap='gray')
plt.title('Loaded')

plt.subplot(n_rows, n_cols, 2)
plt.imshow(image, cmap='gray')
plt.title('Original')

plt.subplot(n_rows, n_cols, 3)
plt.imshow(arr, cmap='gray')
plt.title('Array')

In [ ]:
stats(arr)
stats(np.array(image_loaded))
stats(np.array(image))

## Debug VOI LUT

In [ ]:
image_dicom_vl = dicom_to_np(dicom, voi_lut=True)
image_dicom_vl_u16 = image_dicom_vl.copy().astype(np.uint16)
image_dicom = dicom_to_np(dicom, voi_lut=False)
image_dicom_3 = apply_windowing_custom(dicom.pixel_array.copy(), dicom, dtype=None) # TODO: photometric interp
image_dicom.shape, image_dicom.dtype, image_dicom_vl.dtype, image_dicom_3.dtype

In [ ]:
def stats(arr):
    print(arr.dtype, arr.min(), arr.max(), arr.mean(), arr.std())

In [ ]:
stats(image_png)
stats(image_dicom)
stats(image_dicom_vl)
stats(image_dicom_vl_u16)
stats(image_dicom_3)

In [ ]:
(image_dicom_3 == image_dicom_vl_u16).all()

In [ ]:
n_rows, n_cols = 1, 4
plt.figure(figsize=(14, 5))

plt.subplot(n_rows, n_cols, 1)
plt.title('PNG')
plt.imshow(image_png, cmap='gray')

plt.subplot(n_rows, n_cols, 2)
plt.title('Dicom')
plt.imshow(image_dicom, cmap='gray')

plt.subplot(n_rows, n_cols, 3)
plt.title('Dicom VOI LUT')
plt.imshow(image_dicom_vl, cmap='gray')

plt.subplot(n_rows, n_cols, 4)
plt.title('Dicom VOI LUT uint16')
plt.imshow(image_dicom_vl_u16, cmap='gray')